In [1]:
from __future__ import print_function, division

In [2]:
import numpy as np
import pandas as pd

In [3]:
import os
print('Current directory:', os.getcwd())
import sys
from glob import glob
import zipfile
import json

Current directory: /Users/hmourit/Documents/0project/MScThesis/notebooks


### Constants

In [4]:
results_path = os.path.expanduser('~/Documents/0project/bucket/results')
result_file = 'result_*.json'
block_file = 'results_block_*.json'
zipped_file = 'zipped_results.zip'

### Code

In [5]:
sum(os.path.getsize(f) / 1024 / 1024 for f in glob(os.path.join(results_path, result_file)))

0.1001443862915039

In [6]:
files = glob(os.path.join(results_path, 'result?_*.json'))

In [7]:
def _extract_fields(d, path):
    fields = set()
    for k in d:
        if isinstance(d[k], dict):
            fields |= _extract_fields(d[k], path + k + '.')
        else:
            fields.add(path + k)
    return fields

fields = set()
for f in files:
    results = json.load(open(f, 'rb'))
    if isinstance(results, dict):
        results = [results]
    for experiment in results:
        fields |= _extract_fields(experiment, '')

sorted(fields)        

[u'classifier',
 u'cross_val.n_folds',
 u'cross_val.shuffle',
 u'cross_val.type',
 u'data',
 u'experiment_id',
 u'pca.n_components',
 u'results.accuracy.mean',
 u'results.accuracy.scores',
 u'results.accuracy.std',
 u'results.time',
 u'split.n_iter',
 u'split.test_size',
 u'split.type',
 u'std_select.n_feat',
 u'target',
 u'time']

In [17]:
from Queue import Queue
def extract_json_fields(document):
    q = Queue()
    if isinstance(document, list):
        for d in document:
            q.put(('', d))
    elif isinstance(document, dict):
        q.put(document)
    else:
        raise ValueError("Document doesn't have a valid format.")

    fields = set()
    while not q.empty():
        prefix, d = q.get()
        if isinstance(d, list):
            for dd in d:
                q.put((prefix + '[' if prefix else '', dd))
        elif isinstance(d, dict):
            for key in d:
                q.put((prefix + '.' + key if prefix else key, d[key]))
        else:
            fields.add(prefix)

    return sorted(fields)

In [57]:
files = glob(os.path.join(results_path, 'anova_*.json'))
document = []
error = 0
total = 0
for f in files:
    try:
        document.append(json.load(open(f, 'rb')))
    except:
        error += 1
    total += 1
print('{}/{} errors.'.format(error, total))
extract_json_fields(document)

16/25 errors.


[u'clf',
 u'data',
 u'data_path',
 u'experiments[.iteration',
 u'experiments[.scores[',
 u'experiments[.subsets[.best_params.C',
 u'experiments[.subsets[.best_params.alpha',
 u'experiments[.subsets[.best_params.l1_ratio',
 u'experiments[.subsets[.features[',
 u'experiments[.subsets[.n_features',
 u'experiments[.subsets[.test.y_pred[',
 u'experiments[.subsets[.test.y_true[',
 u'experiments[.subsets[.train.y_pred[',
 u'experiments[.subsets[.train.y_true[',
 u'experiments[.train_samples_idx[',
 u'experiments[.train_samples_label[',
 u'filter',
 u'n_folds',
 u'n_iter',
 u'results_path',
 u'start_time',
 u'target',
 u'test_size',
 u'tissue',
 u'verbose']

In [58]:
from sklearn.metrics import accuracy_score

In [97]:
combinations = set()
for d in document:
    if 'clf' not in d:
        continue
    combinations.add((d['data'], d['tissue'], d['target'], d['filter']))
print(combinations)

set([(u'mdd_raw', None, u'stress', u'anova'), (u'mdd', None, u'stress', u'anova'), (u'mdd_raw37', None, u'stress', u'anova'), (u'mdd', None, u'drug', u'anova')])


In [99]:
scores = {}
data, tissue, target, filter_ = combinations.pop()
for d in document:
    if (d['data'] == data
        and d['tissue'] == tissue
        and d['target'] == target
        and d['filter'] == filter_
        and 'clf' in d):
        for subset in experiment['subsets']:
            size = len(subset['features'])
            if size not in scores:
                scores[size] = defaultdict(list)
            y_true = subset['test']['y_true']
            y_pred = subset['test']['y_pred']
            score = accuracy_score(y_true, y_pred)
            scores[size][d['clf']].append(score)
print(data, tissue, target, filter_)
clfs = sorted({clf for clf in scores[size] for size in scores})
for size in sorted(scores.keys(), reverse=True):
    s = '{:5d} '.format(size)
    for clf in clfs:
        s += '{:1.2f}$\pm${:1.2f} '.format(np.mean(scores[size][clf]), np.std(scores[size][clf]))
    print(s)

mdd None stress anova
37231 0.33$\pm$0.00 0.33$\pm$0.00 
30000 0.33$\pm$0.00 0.33$\pm$0.00 
20000 0.33$\pm$0.00 0.33$\pm$0.00 
10000 0.33$\pm$0.00 0.33$\pm$0.00 
 9000 0.53$\pm$0.00 0.53$\pm$0.00 
 8000 0.33$\pm$0.00 0.33$\pm$0.00 
 7000 0.33$\pm$0.00 0.33$\pm$0.00 
 6000 0.33$\pm$0.00 0.33$\pm$0.00 
 5000 0.33$\pm$0.00 0.33$\pm$0.00 
 4000 0.33$\pm$0.00 0.33$\pm$0.00 
 3000 0.33$\pm$0.00 0.33$\pm$0.00 
 2000 0.53$\pm$0.00 0.53$\pm$0.00 
 1000 0.73$\pm$0.00 0.73$\pm$0.00 
  900 0.33$\pm$0.00 0.33$\pm$0.00 
  800 0.33$\pm$0.00 0.33$\pm$0.00 
  700 0.33$\pm$0.00 0.33$\pm$0.00 
  600 0.53$\pm$0.00 0.53$\pm$0.00 
  500 0.33$\pm$0.00 0.33$\pm$0.00 
  400 0.33$\pm$0.00 0.33$\pm$0.00 
  300 0.33$\pm$0.00 0.33$\pm$0.00 
  200 0.33$\pm$0.00 0.33$\pm$0.00 
  100 0.67$\pm$0.00 0.67$\pm$0.00 
   90 0.53$\pm$0.00 0.53$\pm$0.00 
   80 0.40$\pm$0.00 0.40$\pm$0.00 
   70 0.33$\pm$0.00 0.33$\pm$0.00 
   60 0.40$\pm$0.00 0.40$\pm$0.00 
   50 0.40$\pm$0.00 0.40$\pm$0.00 
   40 0.33$\pm$0.00 0.33$\pm$0.00

In [ ]:
data, tissue, target, filter_, clf = combinations[0]
for d in document

In [68]:
from collections import defaultdict
scores = defaultdict(list)
for d in document:
    for experiment in d['experiments']:
#         print(experiment.keys())
        if 'subsets' not in experiment:
            continue
        for subset in experiment['subsets']:
            size = len(subset['features'])
            y_true = subset['test']['y_true']
            y_pred = subset['test']['y_pred']
            score = accuracy_score(y_true, y_pred)
            scores[(d['data'], d['target'], d['filter'], d['clf'], size)].append(score)

In [74]:
size_score = defaultdict(list)
for k in scores:
    data, target, filter_, clf, size = k
    foo = '{:1.2f}+/-{:1.2f}'.format(np.mean(scores[k]), np.std(scores[k]))
    size_score[size].append(foo)
for k in size_score:
    print(k, size_score[k])

40000 ['0.35+/-0.04', '0.86+/-0.08']
900 ['0.87+/-0.06', '0.33+/-0.03', '0.48+/-0.13', '0.87+/-0.11', '0.91+/-0.06', '0.43+/-0.14', '0.39+/-0.09']
5000 ['0.33+/-0.00', '0.89+/-0.07', '0.91+/-0.07', '0.36+/-0.03', '0.33+/-0.00', '0.33+/-0.00', '0.87+/-0.08']
10 ['0.39+/-0.10', '0.77+/-0.12', '0.36+/-0.07', '0.49+/-0.13', '0.74+/-0.07', '0.44+/-0.16', '0.81+/-0.11']
10000 ['0.89+/-0.07', '0.36+/-0.04', '0.34+/-0.03', '0.91+/-0.06', '0.87+/-0.08', '0.35+/-0.04', '0.37+/-0.07']
7000 ['0.89+/-0.07', '0.91+/-0.07', '0.34+/-0.03', '0.36+/-0.06', '0.41+/-0.11', '0.89+/-0.05', '0.34+/-0.02']
20 ['0.87+/-0.10', '0.41+/-0.10', '0.82+/-0.08', '0.34+/-0.03', '0.83+/-0.09', '0.44+/-0.16', '0.39+/-0.08']
30 ['0.84+/-0.08', '0.85+/-0.10', '0.42+/-0.11', '0.34+/-0.04', '0.39+/-0.11', '0.36+/-0.06', '0.88+/-0.06']
20000 ['0.87+/-0.08', '0.35+/-0.02', '0.86+/-0.09', '0.35+/-0.03', '0.87+/-0.06', '0.34+/-0.02', '0.37+/-0.07']
2000 ['0.87+/-0.08', '0.33+/-0.03', '0.35+/-0.06', '0.91+/-0.06', '0.33+/-0.00',